# This example shows how to run RedeFISH on imaging-based ST data

## Load required packages for RedeFISH

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import RedeFISH

In [2]:
# Set device to 'cuda' if GPU available, otherwise 'cpu'
device='cuda'

## Load scRNA-seq reference and ST data

In [3]:
# The ST data refer to a Mouse Ileum MERFISH dataset
# include three columns (x, y, gene)
st_data = pd.read_csv("example_data/st_data.csv")
st_data


,x,y,gene
0,1705,1271,Maoa
1,1725,1922,Maoa
2,1753,1863,Maoa
3,1760,1865,Maoa
4,1904,794,Maoa
...,...,...,...
819660,5704,38,Hrh1
819661,5685,43,Htr4
819662,5631,61,Taar6
819663,5720,62,Taar7a


In [4]:
# The scRNA-seq data refer to SCP1038 from https://singlecell.broadinstitute.org/single_cell
# We randomly selected 10000 cells for reducing file size of sc_data.h5ad in example_data folder
sc_data = sc.read("example_data/sc_data.h5ad")
sc_data.var_names_make_unique()
sc_data.obs['annotation'].value_counts()

annotation
TA                 3104
Stem cell          2073
Paneth cell        1012
B cell              946
Goblet cell         723
Enterocyte cell     721
T cell              337
Myocyte             172
Neuron              151
Fibroblast          146
Macrophage          133
Endocrine cell      131
Lymphatic           104
Vascular             88
Tuft cell            62
Glia                 51
Mfge8                34
ICC                   8
Microfold cell        4
Name: count, dtype: int64

In [5]:
# Get coordinate of transcriptes and genes
coordinate = np.asarray([st_data['x'], st_data['y']]).T
gene = list(st_data['gene'])

## Estimate Candidate Cell Locations

In [6]:
# Estimate Candidate Cell Locations
# use if __name__ == "__main__": to avoid implementing this function when multiprocessing
if __name__ == "__main__":
    
    model = RedeFISH.center_selection(coordinate, 
                                      prior_count=100, 
                                      subsample=1, 
                                      quantile_cutoff = 0.99,
                                      worker_number=4, 
                                      device=device).to(device)

    estimated_cell_locations = model.train(epochs=1000)
    
    


Estimating Candidate Cell Locations ...

  Initialing data ...
  Epoch: 0 Loss: 0.048934752121567726
  Epoch: 20 Loss: 0.03332716559711844
  Epoch: 40 Loss: 0.023686088430229575
  Epoch: 60 Loss: 0.019870332268998026
  Epoch: 80 Loss: 0.017215412722434847
  Epoch: 100 Loss: 0.014902363598812371
  Epoch: 120 Loss: 0.012777069810777903
  Epoch: 140 Loss: 0.011083917499985546
  Epoch: 160 Loss: 0.009532554323785008
  Epoch: 180 Loss: 0.008266610512277112
  Epoch: 200 Loss: 0.007225707929581404
  Epoch: 220 Loss: 0.006382292398135178
  Epoch: 240 Loss: 0.0056716911832336335
  Epoch: 260 Loss: 0.005082011982565746
  Epoch: 280 Loss: 0.004569826135179028
  Epoch: 300 Loss: 0.004174549013958312
  Epoch: 320 Loss: 0.0038213983796304093
  Epoch: 340 Loss: 0.0035324870003969407
  Epoch: 360 Loss: 0.0032893650371115654
  Epoch: 380 Loss: 0.003055556320992764
  Epoch: 400 Loss: 0.002851004906813614
  Epoch: 420 Loss: 0.0027051096815266648
  Epoch: 440 Loss: 0.002567238706978969
  Epoch: 460 Loss:

## Run RedeFISH for cell alignment and get output files

In [ ]:
model = RedeFISH.alignment(coordinate, 
                              gene,
                              estimated_cell_locations,
                              sc_data,
                              prior_count = 100, 
                              worker_number = 4, 
                              quantile_cutoff = 0.9999,
                              output_dir = "test_output",
                              device = device).to(device)

model.train(epochs=500)



Running RedeFISH ...

Initialing data ...

Cell type distribution of sc/snRNA:
annotation
TA                 1972
Stem cell          1463
Paneth cell         787
Enterocyte cell     674
B cell              638
Goblet cell         560
T cell              243
Endocrine cell      126
Myocyte             123
Neuron              106
Macrophage           99
Fibroblast           90
Vascular             65
Lymphatic            53
Tuft cell            51
Glia                 34
Mfge8                20
ICC                   8
Microfold cell        3
Name: count, dtype: int64
Intersection Gene: 214

Auto select batch size: 16394

  Begin training ...
  Epoch: 1/500 	Region Loss: 0.22748348 	Alignment Loss: 0.5048994
  Epoch: 2/500 	Region Loss: 0.02057054 	Alignment Loss: 0.49913695
  Epoch: 3/500 	Region Loss: 0.0120572 	Alignment Loss: 0.49479634
  Epoch: 4/500 	Region Loss: 0.02191962 	Alignment Loss: 0.49182826
  Epoch: 5/500 	Region Loss: 0.03643392 	Alignment Loss: 0.5240609
  Epoch: 6/50

#### Parameter for RedeFISH alignment ####

coordinate : Numpy Array, required
    The coordinate of transcripts. dimention: [Transcrips number, 2]
            
gene : list, required
    Gene name of transcrips

candidate_cell_center : Numpy Array, required
    Coordinates of candidate cell centers. This can be obtained by center_selection
    program.

sc_data : Scanpy AnnData, required
    Scanpy AnnData of single cell reference. Must include annotation information.
    Thus: sc_data.obs.annotation

prior_count : int, default=100 (important)
    Prior estimate of number of transcripts in each cells. No need to be precise

number_min_genes : int, default=10
    Minimun number of genes for single cells.
    sc.pp.filter_cells(sc_data, min_genes=number_min_genes)

number_cell_state : int, default=10
    Number of cell state for each cell type reference.

cell_count_cutoff : int, AutoSelect
    Minimun number of transcripts in each aligned cells.
    default to be prior_count/5.

k_number : int, default=100
    k_number for KNN-graph.

k_number_for_distance: int, default=8
    k_number for calculating distance score

quantile_cutoff : float, default=0.999
    Cutoff of the quantile function for distance distribution. Used for calculating 
    distance score

alphashape_value : int, default=5
    Alpha value for running alphashape program.

top_N_single_cell: int, default=20
    Number of single cell used for cell type inference and expression prediction.

worker_number : int, default=1
    Number of CPU cores.

output_dir : str, default="./"
    output directory 

device : str, default='cpu'
    Set device for running alignment.